#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m-%d')

#Agora vamos importar nosso arquivo de PIB brasileiro
df = pd.read_csv('..\Data_FS_04.24\PIB.csv')
print(df.head(10))

      PIB        Date
0  189323  1996-03-31
1  204611  1996-06-30
2  221513  1996-09-30
3  239316  1996-12-31
4  219117  1997-03-31
5  232890  1997-06-30
6  246178  1997-09-30
7  253904  1997-12-31
8  235701  1998-03-31
9  251936  1998-06-30


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PIB     112 non-null    int64 
 1   Date    112 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [4]:
#df.loc[:, 'Date'] = pd.to_datetime(df['Date']).date
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   PIB     112 non-null    int64         
 1   Date    112 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.9 KB


### Avaliando os dados historicos do PIB Brasileiro

In [6]:
df.set_index('Date',inplace=True)

In [7]:
df=df.sort_index(axis=0)
print(df.head(10))

               PIB
Date              
1996-03-31  189323
1996-06-30  204611
1996-09-30  221513
1996-12-31  239316
1997-03-31  219117
1997-06-30  232890
1997-09-30  246178
1997-12-31  253904
1998-03-31  235701
1998-06-30  251936


In [8]:
len(df)

112

In [9]:
df1=df[:56]
df2=df[100:]
df2.head()


,PIB
Date,
2021-03-31,2156670
2021-06-30,2203639
2021-09-30,2295851
2021-12-31,2355982
2022-03-31,2319528


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [10]:
exp = TSForecastingExperiment()
exp.setup(data=df2, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,PIB
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(12, 1)"
5,Transformed data shape,"(12, 1)"
6,Transformed train set shape,"(11, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [14]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

In [ ]:
#exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":4})

## Checking Train-Test Split

In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [11]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

### xgboost_cds_dt

In [12]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(xgboost_cds_dt, fh = 8)


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2022Q4,0.2887,0.2416,23005.6417,23005.6417,0.0089,0.0089
1,2023Q1,0.8958,0.7548,69032.1866,69032.1866,0.0256,0.0252
2,2023Q2,0.6144,0.5290,50167.9120,50167.9120,0.0183,0.0181
Mean,NaT,0.5996,0.5085,47401.9135,47401.9135,0.0176,0.0174
SD,NaT,0.2480,0.2100,18891.7755,18891.7755,0.0068,0.0067


## exp_smooth  Exponential Smoothing 

In [1]:
exp_smooth  = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
#exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
#               data_kwargs = {'fh' : 8},
#               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(exp_smooth, fh = 8)


NameError: name 'exp' is not defined

## huber_cds_dt    Huber w/ Cond. Deseasonalize & Detrending   

In [ ]:

huber_cds_dt  = exp.create_model('huber_cds_dt ')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = huber_cds_dt , 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(huber_cds_dt , fh = 8)


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### arima   ARIMA 

In [ ]:

arima = exp.create_model('arima')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(arima, fh = 8)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,1.445811e+06
2023,1.376939e+06
2024,1.308067e+06
2025,1.239196e+06
2026,1.170324e+06
2027,1.101452e+06
2028,1.032580e+06
2029,9.637086e+05
2030,8.948369e+05


In [ ]:
## AUTO_ARIMA

In [ ]:
auto_arima = exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = auto_arima, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(auto_arima, fh = 8)

## ALL GRAPHS

In [ ]:
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

               

NameError: name 'huber_cds_dt' is not defined